In [15]:
from langchain.chat_models import ChatOpenAI
# promptTemplate은 그냥 string을 이용해서 template을 만들고 ChatPromptTemplate은 template을 message로 부터 만든다.
from langchain.prompts import PromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

#값이 높아질수록 더 많은 창의성과 무작위성을 갖게된다.
chat = ChatOpenAI(
  temperature=0.1, 
  #응답이 끝날때까지 기다리는 것이 아니라 모델의 응답이 생성되는것을 볼 수 있게 해준다.
  streaming=True,
  #문자가 생길때마다 print 해줘서 console에서 응답의 진행을 볼 수 있음
  callbacks=[
    StreamingStdOutCallbackHandler()
  ]
)

#단순 텍스트 predict
chat.predict("...")

template = PromptTemplate.from_template("What is the distance between {country_a} and {country_b}.")

# template 위쪽에는 개발자들에게 필요한 일종의 검증계층이 있어 실수를 줄일 수 있음
#template을 format 하여 Prompt에 저장
prompt = template.format(country_a="Mexico", country_b="Thailand" )

chat.predict(prompt)


Hello! How can I assist you today?The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.

'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [5]:
from langchain.schema import HumanMessage, AIMessage, SystemMessage

#SystemMessage is sent by the system, the AIMessage is sent by the AI and the HumanMessage is sent by the user. They all make up a conversation.
message=[
  #systemMessage-AI에 일종의 기본 설정, 기본 값, 기본 context 설정, LLM에 설정들을 제공하기 위한 메세지
  SystemMessage(content="You are a geography expert, And you only reply in Italian"),
  #이미 존재하는 가상 대화 설정, AI가 답변을 한것으로 가정
  #값을 메모리에 추가
  AIMessage(content="Ciao, michiamo Palolo!"),
  #사용자로서 질문
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

# predict message
chat.predict_messages(message)


Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come mi chiamo? Mi chiamo Palolo!

AIMessageChunk(content='Ciao! La distanza tra il Messico e la Thailandia è di circa 16.000 chilometri. Come mi chiamo? Mi chiamo Palolo!')

In [6]:
from langchain.prompts import ChatPromptTemplate

#more customizable
template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert, And you only reply in {language}"),
  ("ai", "Ciao, michiamo {name}!"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

#validate variables
prompt = template.format_messages(language="Greek", name="Socrates", country_a="Mexico", country_b="Thailand")

chat.predict_messages(prompt)



Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.

AIMessageChunk(content='Γεια σας! Το όνομά μου είναι Σωκράτης. Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα.')

In [7]:
# 확장해서 outputPaser 구현
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
      #strip-text 앞뒤 공백 자름
      items = text.strip().split(",")
      return list(map(str.strip,items))
    
p = CommaOutputParser()

p.parse("Hello, how, are, you")



['Hello', 'how', 'are', 'you']

In [8]:
template = ChatPromptTemplate.from_messages([
  ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. DoNOT reply with anything else."),
  ("human", "{question}")
])

prompt = template.format_messages(max_items=10, question="What are the planets?")

result = chat.predict_messages(prompt)

p = CommaOutputParser()

p.parse(result.content)

mercury, venus, earth, mars, jupiter, saturn, uranus, neptune

['mercury', 'venus', 'earth', 'mars', 'jupiter', 'saturn', 'uranus', 'neptune']

In [9]:
# | 기호를 통해 chain 생성
# langchain이 .format_message호출하고, chat.predict 호출하고, parse를 호출한다.
chain = template | chat | CommaOutputParser()

#chain_two = template_2 | chat | outputParser
#all = chain | chain_two | outputParser

chain.invoke({
  "max_items":5,
  "question":"what are th pokemons?"
})

pikachu, bulbasaur, charmander, squirtle, jigglypuff

['pikachu', 'bulbasaur', 'charmander', 'squirtle', 'jigglypuff']

In [10]:
chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipes for any type of cuisine with easy to find ingredients."), ("human", "I want to cook {cuisine} food,")
])

chef_chain = chef_prompt | chat

In [11]:
veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
  ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

#runnable map-chef_chain을 먼저 실행하고, 그 결과를 veg_chain으로 전달하도록 한다.
final_chain = { "recipe" : chef_chain } | veg_chain 

final_chain.invoke({
  "cuisine":"indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Chicken Tikka Masala

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1 tablespoon ginger, minced
- 1 can (14 oz) tomato sauce
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- Salt and pepper, to taste
- Fresh cilantro, for garnish
- Cooked rice or naan, for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the chicken pieces and marinate for at least 1 hour, or overnight for best results.

2. Preheat the oven to 400°F (

AIMessageChunk(content="For a vegetarian version of Chicken Tikka Masala, we can replace the chicken with a plant-based alternative such as tofu or paneer. Here's how you can adapt the recipe:\n\n**Vegetarian Tikka Masala**\n\n**Ingredients:**\n- 1 block of firm tofu or paneer, cut into bite-sized cubes\n- 1 cup plain yogurt (you can use dairy-free yogurt for a vegan version)\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1 tablespoon ginger, minced\n- 1 can (14 oz) tomato sauce\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- Salt and pepper, to taste\n- Fresh cilantro, for garnish\n- Cooked rice or naan, for serving\n\n**Instructions:**\n1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the minced ginger, and a pinch of salt. Add the tofu or paneer cubes and marinate

In [18]:

#FewShot- 모델에게 더 나은 대답을 할 수 있도록 하는 예제들을 준다
from langchain.prompts.few_shot import FewShotPromptTemplate

examples = [
  {
    "question":"What do you know about France?",
    "answer": """Here is what I know: 
    Capital:paris
    Language:French
    Food:Wine and Cheese
    Currency:Euro
    """,
  },
  {
    "question":"What do you know about Italy?",
    "answer": """Here is what I know: 
    Capital:Rome
    Language:Italian
    Food:Pizza and Pasta
    Currency:Euro
    """,
  },
    {
    "question":"What do you know about Greece?",
    "answer": """Here is what I know: 
    Capital:Athene
    Language:Greek
    Food:Souvlaki adn Feta Cheese
    Currency:Euro
    """,
  }
]

example_template = """
  Human:{question}
  AI:{answer}
"""

example_prompt = PromptTemplate.from_template(example_template)

prompt = FewShotPromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
  #형식화된 모든 예제 마지막에 나오는 내용
  suffix="Human:What do you know about {country}",
  input_variables=["country"]
)

#prompt.format(country="Germany")
chain = prompt | chat

chain.invoke({
  "country":"Germany"
})

AI: Here is what I know: 
    Capital: Berlin
    Language: German
    Food: Bratwurst and Sauerkraut
    Currency: Euro

AIMessageChunk(content='AI: Here is what I know: \n    Capital: Berlin\n    Language: German\n    Food: Bratwurst and Sauerkraut\n    Currency: Euro')

In [20]:

from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.prompts import ChatPromptTemplate

examples = [
  {
    "country":"France",
    "answer": """Here is what I know: 
    Capital:paris
    Language:French
    Food:Wine and Cheese
    Currency:Euro
    """,
  },
  {
    "country":"Italy",
    "answer": """Here is what I know: 
    Capital:Rome
    Language:Italian
    Food:Pizza and Pasta
    Currency:Euro
    """,
  },
    {
    "country":"Greece",
    "answer": """Here is what I know: 
    Capital:Athene
    Language:Greek
    Food:Souvlaki adn Feta Cheese
    Currency:Euro
    """,
  }
]

example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know about {country}?"),
  ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
  example_prompt=example_prompt,
  examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a geography expert, you give short answers"),
  example_prompt,
  ("human","What do you know about {country}")
])

#prompt.format(country="Germany")
chain = final_prompt | chat

chain.invoke({
  "country":"Germany"
})

Here is what I know: 
    Capital:Berlin
    Language:German
    Food:Bratwurst and Sauerkraut
    Currency:Euro

AIMessageChunk(content='Here is what I know: \n    Capital:Berlin\n    Language:German\n    Food:Bratwurst and Sauerkraut\n    Currency:Euro')